In [1]:
import os
import glob
import polars as pl
import time

In [2]:

class FarcasterLinks:
    def __init__(self):
        self.download_path = "/Users/joaovictorassisdasilveira/Desktop/work/bleu/repositories/op-farcaster-social-graph/repo/farcaster-social-graph/notebooks/parquet/full"

    def get_links_lazy_df(self, file_path):
        return pl.scan_parquet(file_path)

    def get_undirected_links(self, links_df):
        """
        """
        df_filtered = links_df.filter(
            (pl.col("deleted_at").is_null()) & 
            (pl.col("fid") != pl.col("target_fid")) & 
            (pl.col("type") == "follow")
        ).select([
            "fid", "target_fid"
        ])


        df_sorted = df_filtered.with_columns([
            pl.min_horizontal(["fid", "target_fid"]).alias("sorted_fid"),
            pl.max_horizontal(["fid", "target_fid"]).alias("sorted_target_fid")
        ])
        
        df_grouped = df_sorted.group_by(["sorted_fid", "sorted_target_fid"]).agg(
            pl.count().alias("count")
        )
        
        mutual_df = df_grouped.filter(pl.col("count") == 2).select([
            pl.col("sorted_fid").alias("fid"),
            pl.col("sorted_target_fid").alias("target_fid")
        ])

        reversed_mutual_df = mutual_df.select([
            pl.col("target_fid").alias("fid"),
            pl.col("fid").alias("target_fid")
        ])
        
        df_concat = pl.concat([mutual_df, reversed_mutual_df])

        # return df_filtered
        
        # Perform the self-join to find mutual links
        # mutual_links = df_filtered.join(
        #     df_filtered,
        #     left_on=["fid", "target_fid"],
        #     right_on=["target_fid", "fid"],
        #     suffix="_reverse"
        # )
        
        return df_concat

    # def get_parquet_schema(self):
    #     schema = {
    #         "fid": pl.Int64,
    #         "target_fid": pl.Int64,
    #         "hash": pl.Binary,
    #         "timestamp": pl.Datetime("ns"),
    #         "created_at": pl.Datetime("ns"),
    #         "updated_at": pl.Datetime("ns"),
    #         "deleted_at": pl.Datetime("ns"),
    #         "type": pl.Utf8,
    #         "display_timestamp": pl.Datetime("ns"),
    #         "id": pl.Int64
    #     }
    #     return schema
    
    def write_links_to_parquet(self, links_df):
        filename = f"/processed-farcaster-mutual-links-{int(time.time())}.parquet"
        links_df.sink_parquet(self.download_path + filename)
    

    def execute(self):
        start = time.time()
        links_file_path = "/Users/joaovictorassisdasilveira/Desktop/work/bleu/repositories/op-farcaster-social-graph/repo/farcaster-social-graph/notebooks/parquet/full/farcaster-links-0-1730134800.parquet"
        links_lazy_df = self.get_links_lazy_df(links_file_path)
        mutual_links = self.get_undirected_links(links_lazy_df)

        # links =  await mutual_links.collect_async()
        # links =  mutual_links.collect()
        
        # self.write_links_to_parquet(mutual_links)
        end = time.time()
        
        print(f"Execution time: {end - start} seconds")

        return mutual_links


In [3]:
# pl.Config.set_global_thread_pool_size(4)
data = FarcasterLinks()
links = data.execute()

Execution time: 0.00028061866760253906 seconds


In [ ]:
%%time
links.sink_parquet("mutual_connections.parquet")

In [10]:
df_filtered_sorted = links.sort(["fid", "target_fid"])


In [12]:
df_filtered_sorted = df_filtered_sorted.lazy()

In [11]:
# mutual_links = df_filtered_sorted.join(
#     df_filtered_sorted,
#     left_on=["fid", "target_fid"],
#     right_on=["target_fid", "fid"],
#     suffix="_reverse"
# )

KeyboardInterrupt: 

In [16]:
# # df = links.lazy()

# # Perform the self-join to find mutual connections
# mutual_df = df_filtered_sorted.join(
#     df_filtered_sorted, 
#     left_on=["fid", "target_fid"], 
#     right_on=["target_fid", "fid"],
#     how="inner"
# ).select([
#     pl.col("fid"), 
#     pl.col("target_fid")
# ])

In [4]:
df_sorted = links.with_columns([
    pl.min_horizontal(["fid", "target_fid"]).alias("sorted_fid"),
    pl.max_horizontal(["fid", "target_fid"]).alias("sorted_target_fid")
])

# Step 2: Group by the sorted pairs and count occurrences
df_grouped = df_sorted.group_by(["sorted_fid", "sorted_target_fid"]).agg(
    pl.count().alias("count")
)
# Step 3: Filter to keep only pairs with exactly 2 occurrences (mutual connections)
mutual_df = df_grouped.filter(pl.col("count") == 2).select([
    pl.col("sorted_fid").alias("fid"),
    pl.col("sorted_target_fid").alias("target_fid")
])
# Since this is a large dataset, save or stream the result rather than collecting into memory
# mutual_df.sink_parquet("mutual_connections.parquet")

/var/folders/v9/zdrfs19x765_sy8y01lgc13m0000gn/T/ipykernel_47861/3096096080.py:8: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("count")


In [5]:
%%time
# start = time.time()
# res = mutual_df.collect(streaming=True)
mutual_df.sink_parquet("mutual_connections.parquet")
# mutual_df.explain(streaming=True)
# 
# end = time.time()
# print(f"Execution time: {end - start} seconds")

CPU times: user 39.2 s, sys: 18.5 s, total: 57.7 s
Wall time: 27.4 s


In [10]:
pl.thread_pool_size()
dir(pl)

['Array',
 'ArrowError',
 'Binary',
 'Boolean',
 'Categorical',
 'CategoricalRemappingWarning',
 'ChronoFormatWarning',
 'ColumnNotFoundError',
 'ComputeError',
 'Config',
 'DATETIME_DTYPES',
 'DURATION_DTYPES',
 'DataFrame',
 'DataType',
 'Date',
 'Datetime',
 'Decimal',
 'DuplicateError',
 'Duration',
 'Enum',
 'Expr',
 'FLOAT_DTYPES',
 'Field',
 'Float32',
 'Float64',
 'INTEGER_DTYPES',
 'InProcessQuery',
 'Int16',
 'Int32',
 'Int64',
 'Int8',
 'InvalidOperationError',
 'LazyFrame',
 'List',
 'MapWithoutReturnDtypeWarning',
 'NESTED_DTYPES',
 'NUMERIC_DTYPES',
 'NoDataError',
 'Null',
 'Object',
 'OutOfBoundsError',
 'PolarsDataType',
 'PolarsError',
 'PolarsPanicError',
 'PolarsWarning',
 'SQLContext',
 'SchemaError',
 'SchemaFieldNotFoundError',
 'Series',
 'ShapeError',
 'String',
 'StringCache',
 'Struct',
 'StructFieldNotFoundError',
 'TEMPORAL_DTYPES',
 'Time',
 'UInt16',
 'UInt32',
 'UInt64',
 'UInt8',
 'Unknown',
 'UnstableWarning',
 'Utf8',
 '__all__',
 '__annotations__',
 

In [4]:
%%time
# df_filtered = links.filter(
#     (pl.col("deleted_at").is_null()) &
#     (pl.col("fid") != pl.col("target_fid")) &
#     (pl.col("type") == "follow")
# ).select(["fid", "target_fid"])  # Only keep the necessary columns

# Create a reverse relationship (swapping fid and target_fid)
df_reversed = links.select([
    pl.col("target_fid").alias("fid"),
    pl.col("fid").alias("target_fid")
])

# Concatenate the original and reversed DataFrames
df_concat = pl.concat([links, df_reversed])

# Group by fid and target_fid and count occurrences (mutual follows will have count = 2)
mutual_links = (
    df_concat
    .group_by(["fid", "target_fid"])
    .agg([
        pl.count().alias("count")
    ])
    .filter(pl.col("count") == 2)  # Keep only mutual follows
    .select(["fid", "target_fid"])  # Final output
)

mutual_links.sink_parquet("mutual_connections2.parquet")
# slower
test = pl.read_parquet("mutual_connections2.parquet")
test

<timed exec>:21: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.


CPU times: user 1min 14s, sys: 25.1 s, total: 1min 39s
Wall time: 1min


In [6]:
mutual = pl.read_parquet("mutual_connections.parquet")
mutual

fid,target_fid
i64,i64
503739,509383
401273,426502
344985,574595
665654,665741
450820,498037
…,…
548559,574781
640561,859377
628358,696668


In [10]:
# test.filter((pl.col('fid')==509383)&(pl.col('target_fid')==503739))

fid,target_fid
i64,i64


In [78]:
%%time
fids = mutual.lazy().select("fid").unique()
target_fids = mutual.lazy().select("target_fid").unique().rename({"target_fid": "fid"})
all_fids = pl.concat([fids, target_fids]).unique().collect()  # Collect to make sure the concatenation works

mutual_reindex = all_fids.with_columns(
    pl.arange(0, all_fids.shape[0]).alias("index")  # Access shape[0] after collecting
).lazy()

mutual_links_with_index = mutual.lazy().join(
    mutual_reindex.select([pl.col('fid'), pl.col('index').alias('fid_index')]),
    on='fid',
    how='left'
).join(
    mutual_reindex.select([pl.col('fid'), pl.col('index').alias('target_fid_index')]),
    left_on='target_fid',
    right_on='fid',
    how='left'
)

# Reverse the links and align the schema by renaming columns
df_reversed = mutual_links_with_index.select([
    pl.col("target_fid").alias("fid"),
    pl.col("fid").alias("target_fid"),
    pl.col("target_fid_index").alias("fid_index"),
    pl.col("fid_index").alias("target_fid_index"),
])

order = ["fid", "target_fid", "fid_index", "target_fid_index"]
# Use lazy execution and ensure schemas match before concatenating
mutual_links_with_index_concatenated = pl.concat([
    mutual_links_with_index.select(order),  # Ensure column names match
    df_reversed.select(order)  # Ensure column names match
])

# mutual_links_with_index_concatenated.

mutual_links_with_index_concatenated = mutual_links_with_index_concatenated.with_columns(
    (pl.col("fid_index").cast(pl.Utf8) + " " + pl.col("target_fid_index").cast(pl.Utf8)).alias("connection")
)

bot_or_not = pl.scan_parquet(f"/Users/joaovictorassisdasilveira/Desktop/work/bleu/repositories/op-farcaster-social-graph/repo/farcaster-social-graph/notebooks/parquet/full/bot_or_not_without_info.parquet")

mutual_links_with_bot_information = mutual_links_with_index_concatenated.join(bot_or_not, how="left", on="fid")
mutual_links_with_bot_information.sink_parquet("testei.parquet")

<timed exec>:9: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
<timed exec>:42: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.


CPU times: user 31.1 s, sys: 2.08 s, total: 33.2 s
Wall time: 9.29 s


In [80]:
%%time
connections_list = mutual_links_with_bot_information.select("connection").collect(streaming=True).to_series().to_list()

with open('graph.txt', 'w') as f:
    f.write("\n".join(connections_list))

CPU times: user 14.9 s, sys: 3.05 s, total: 17.9 s
Wall time: 10.3 s


In [39]:
mutual_links_with_bot_information.filter(pl.col("connection").is_null()).cou

fid,target_fid,fid_index,target_fid_index,connection,bot
i64,i64,i64,i64,str,bool
567046,809753,79184,null,null,null
754556,754608,142173,null,null,null
639968,734886,48701,null,null,null
243192,305891,114685,null,null,null
299062,415776,241764,null,null,null
…,…,…,…,…,…
16228,1356,null,121613,null,null
579022,574150,null,116207,null,null
813676,570528,null,249510,null,null


In [84]:
con = pl.scan_parquet("processed-farcaster-undirected-connections-1728923932.parquet")

In [86]:
con.collect()

fid_index,target_fid_index,bot
i64,i64,bool
61552,352612,false
279002,62892,null
193624,157311,null
365755,218793,false
300964,223801,null
…,…,…
316096,35857,null
41544,137984,null
303571,361575,null


In [108]:
%%time
import numpy as np

connections_df = con.select(["fid_index", "target_fid_index"]).collect(streaming=True)
connections_generator = ((row[0], row[1]) for row in connections_df.iter_rows())

CPU times: user 1.16 s, sys: 1.14 s, total: 2.3 s
Wall time: 647 ms


In [116]:
sybils_df = con.filter(pl.col("bot") == False).select("fid_index").unique().collect(streaming=True)
sybils_df.sort('fid_index')

fid_index
i64
10
130
131
138
317
…
367159
367204
367229


In [110]:
import sys
# sys.getsizeof(connections)

list_size_in_bytes = sys.getsizeof(connections_df)
list_size_in_bytes / (1024 * 1024)



4.57763671875e-05

In [107]:
connections_array[0]

array([ 61552, 352612])

In [99]:
con.collect()

fid_index,target_fid_index,bot
i64,i64,bool
61552,352612,false
279002,62892,null
193624,157311,null
365755,218793,false
300964,223801,null
…,…,…
316096,35857,null
41544,137984,null
303571,361575,null


In [101]:
con.unique(subset=["fid_index","target_fid_index"]).collect()

fid_index,target_fid_index,bot
i64,i64,bool
348894,236096,false
221592,300871,null
177021,278043,null
238961,43829,true
21810,184242,null
…,…,…
37843,172503,null
48119,218739,null
155850,82405,false


In [1]:
import psutil
print(f'thread count per core: {psutil.cpu_count() // psutil.cpu_count(logical=False)}')

thread count per core: 1
